In [6]:
import pandas as pd
import json

In [36]:
df = pd.read_csv("sales_orders.csv", parse_dates=['order_date'])
df['line_items'] = df['line_items'].apply(json.loads)
df.head()

,order_number,order_date,line_items,fulfillment
0,387005,2016-01-22,[{'product': {'product_name': 'MGS Age of Empi...,In store
1,395005,2016-01-30,[{'product': {'product_name': 'Contoso DVD 60 ...,In store
2,423011,2016-02-27,[{'product': {'product_name': 'Contoso 16GB Mp...,In store
3,600006,2016-08-22,[{'product': {'product_name': 'Contoso Water H...,Online
4,652002,2016-10-13,[{'product': {'product_name': 'Contoso Genuine...,In store


In [37]:
df = df.explode('line_items')
df

,order_number,order_date,line_items,fulfillment
0,387005,2016-01-22,{'product': {'product_name': 'MGS Age of Empir...,In store
0,387005,2016-01-22,{'product': {'product_name': 'A. Datum Bridge ...,In store
0,387005,2016-01-22,{'product': {'product_name': 'WWI Desktop PC1....,In store
1,395005,2016-01-30,{'product': {'product_name': 'Contoso DVD 60 D...,In store
1,395005,2016-01-30,{'product': {'product_name': 'SV DVD 38 DVD St...,In store
...,...,...,...,...
98,2175000,2020-12-14,{'product': {'product_name': 'The Phone Compan...,In store
98,2175000,2020-12-14,{'product': {'product_name': 'The Phone Compan...,In store
98,2175000,2020-12-14,{'product': {'product_name': 'Fabrikam Laptop1...,In store
98,2175000,2020-12-14,{'product': {'product_name': 'SV DVD 14-Inch P...,In store


In [40]:
df_flattened = (
    pd.json_normalize(df["line_items"], sep='_')
    .rename({"product_product_name" : "product_name", "product_product_price" : "product_price" }, axis='columns')
)
df_flattened

,quantity,product_name,product_price
0,3,MGS Age of Empires II Gold Edition2009 E172,32.00
1,2,A. Datum Bridge Digital Camera M300 Pink,186.90
2,1,WWI Desktop PC1.80 E1801 Silver,269.90
3,5,Contoso DVD 60 DVD Storage Binder L20 Black,22.89
4,5,SV DVD 38 DVD Storage Binder E25 Silver,9.99
...,...,...,...
225,2,The Phone Company Smart phones without camera ...,129.00
226,4,The Phone Company PDA Phone Unlocked 3.7 inche...,238.00
227,3,Fabrikam Laptop10.1 M0101 Silver,384.90
228,4,SV DVD 14-Inch Player Portable L100 White,259.99


In [47]:
result = pd.concat(
    [df.drop(columns=['line_items']).reset_index(drop=True), df_flattened], axis=1
)
result

,order_number,order_date,fulfillment,quantity,product_name,product_price
0,387005,2016-01-22,In store,3,MGS Age of Empires II Gold Edition2009 E172,32.00
1,387005,2016-01-22,In store,2,A. Datum Bridge Digital Camera M300 Pink,186.90
2,387005,2016-01-22,In store,1,WWI Desktop PC1.80 E1801 Silver,269.90
3,395005,2016-01-30,In store,5,Contoso DVD 60 DVD Storage Binder L20 Black,22.89
4,395005,2016-01-30,In store,5,SV DVD 38 DVD Storage Binder E25 Silver,9.99
...,...,...,...,...,...,...
225,2175000,2020-12-14,In store,2,The Phone Company Smart phones without camera ...,129.00
226,2175000,2020-12-14,In store,4,The Phone Company PDA Phone Unlocked 3.7 inche...,238.00
227,2175000,2020-12-14,In store,3,Fabrikam Laptop10.1 M0101 Silver,384.90
228,2175000,2020-12-14,In store,4,SV DVD 14-Inch Player Portable L100 White,259.99


In [55]:
online_orders = result[result['fulfillment'] == 'Online']
online_orders["profit"] = online_orders.quantity * online_orders.product_price
online_orders

C:\Users\isaac\AppData\Local\Temp\ipykernel_51428\379842513.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  online_orders["profit"] = online_orders.quantity * online_orders.product_price


,order_number,order_date,fulfillment,quantity,product_name,product_price,profit
6,600006,2016-08-22,Online,1,Contoso Water Heater 2.6GPM E0900 Green,508.00,508.00
7,600006,2016-08-22,Online,1,The Phone Company Pen Touch Screen Phones-2.6 ...,290.00,290.00
8,600006,2016-08-22,Online,6,Contoso USB Wave Multi-media Keyboard E280 Black,15.90,95.40
9,600006,2016-08-22,Online,7,SV 16xDVD M320 Black,119.00,833.00
22,755003,2017-01-24,Online,2,Proseware Laptop15.4W M518 Black,758.00,1516.00
23,755003,2017-01-24,Online,2,Contoso Water Heater 1.5GPM E0800 Grey,257.50,515.00
24,755003,2017-01-24,Online,5,Contoso Expandable 2-Handset Cordless Phone Sy...,40.19,200.95
33,1023001,2017-10-19,Online,5,WWI 2GB Pulse Smart pen M100 Black,199.95,999.75
34,1023001,2017-10-19,Online,2,A. Datum Super-zoom Digital Camera X300 Green,290.00,580.00
41,1121013,2018-01-25,Online,1,Contoso 16GB Mp5 Player M1600 Red,199.90,199.90


In [56]:
online_orders['profit'].sum()

18238.979999999996